<div class="alert alert-block alert-info">
<ul>
<li>Course: Machine Learning</li>
<li>Project: 01</li>
<li>Team members</li>
<li>Mauro Travieso Pena</li>
<li>Quoc Huy Luong</li>
<li>Ngoc Bao Tran</li>
<ul>
</div>



# Classification with Decision Tree

<div class="alert alert-block alert-info">
Some comments here 
</div>


# Import Libraries 

In [ ]:
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import import_ipynb 
import functions as myLib


# Load data from files

In [ ]:
RawData = pd.read_csv('../data/anime.csv')


#### Clean and Transform Data

In [ ]:
Cleaned_Data = myLib.myCleanAndTransformData(RawData)
Y_Data = Cleaned_Data['Class']
X_Data = Cleaned_Data.drop(columns=['Class'])


#### Normalize  Data

In [ ]:
Normalized_Data = myLib.myNormalize(X_Data)

#### PCA

In [ ]:
n_components=10
PCA_Data = myLib.myPCA(Normalized_Data,n_components)
PCA_Data.head()


####----------------------------------------------------------------
# Decision Tree Classification 
####----------------------------------------------------------------

In [ ]:
Y_Data.head()

In [ ]:
X_Data.head()

In [ ]:
Normalized_Data.head()

In [ ]:
PCA_Data.head()


####----------------------------------------------------------------
#### Split Original Data
####----------------------------------------------------------------

In [ ]:
X_train, X_test, y_train, y_test  = myLib.mySplitData(X_Data,Y_Data,0.33,42)

X_train.head()

In [ ]:
X_test.head()

In [ ]:
y_train.head()

In [ ]:
y_test.head()


####----------------------------------------------------------------
#### Split Original Normalized_Data
####----------------------------------------------------------------

In [ ]:
Normalized_X_train, Normalized_X_test, Normalized_y_train, Normalized_y_test  = myLib.mySplitData(Normalized_Data,Y_Data,0.33,42)

Normalized_X_train.head()

In [ ]:
Normalized_X_test.head()

In [ ]:
Normalized_y_train.head()

In [ ]:
Normalized_y_test.head()


####----------------------------------------------------------------
#### Split  PCA_Data
####----------------------------------------------------------------

In [ ]:
PCA_X_train, PCA_X_test, PCA_y_train, PCA_y_test  = myLib.mySplitData(PCA_Data,Y_Data,0.33,42)

PCA_X_train.head()

In [ ]:
PCA_X_test.head()

In [ ]:
PCA_y_train.head()

In [ ]:
PCA_y_test.head()


####----------------------------------------------------------------
# Model Training 
####----------------------------------------------------------------

In [22]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

def myDT(clf,X_train,y_train,X_test,y_test,cv):
    #Fit the model
    #The maximum depth of the tree. 
    #If None, then nodes are expanded until all leaves are pure
    # or until all leaves contain less than min_samples_split samples.
    #min_samples_split: by default = 1
    clf.fit(X_train, y_train)
    #check accuracy of model on the test data
    score = clf.score(X_test, y_test)
    #Using CV
    cv_scores = cross_val_score(clf, X_train, y_train, cv=cv)
    return score,cv_scores,[clf.get_n_leaves(),clf.get_depth()]


array([0.77477989, 0.76285146, 0.78159614, 0.76505682, 0.78977273,
       0.77755682, 0.70880682, 0.71789773, 0.64791134, 0.37283319])

In [34]:
def myDT_ModelTurning(clf,depths,X_train,y_train,X_test,y_test,cv):
    df = pd.DataFrame({"depth":[],"training_score":[],"cv_score":[]})
    for depth in depths:
        clf1 = DecisionTreeClassifier(max_depth=depth,random_state=0)
        score,cv_scores,params = myDT(clf1,X_train,y_train,X_test,y_test,cv)
        df = df.append(pd.DataFrame({"depth":depth,"training_score":[score],"cv_score":[cv_scores.mean()]}),ignore_index=True)
    return df

In [ ]:

cv = 10
clf = DecisionTreeClassifier(random_state=0)

#Original Data
score,cv_scores,params = myDT(clf,X_train,y_train,X_test,y_test,cv)
score

In [ ]:
cv_scores.mean()

In [ ]:
params

In [ ]:
#Normalized Data
score,cv_scores,params = myDT(clf,Normalized_X_train,Normalized_y_train,Normalized_X_test,Normalized_y_test,cv)
score

In [ ]:
cv_scores.mean()

In [ ]:
params


In [ ]:
#PCA Data
score,cv_scores,params = myDT(clf,PCA_X_train,PCA_y_train,PCA_X_test,PCA_y_test,cv)
score

In [ ]:
cv_scores.mean()

In [ ]:
params



####----------------------------------------------------------------
# Learning Curve
####----------------------------------------------------------------
#https://scikit-learn.org/stable/modules/learning_curve.html

In [ ]:
def myDT_LearningCurve(clf,X_Data,Y_Data,splits,cv):
    df = pd.DataFrame({"percent":[],"training_score":[],"cv_score":[]})
    for X in splits:
        X_train, X_test, y_train, y_test  = myLib.mySplitDataByTrainSize(X_Data,Y_Data,X,42)
        score,cv_scores,params = myDT(clf,X_train,y_train,X_test,y_test,cv)
        df = df.append(pd.DataFrame({"percent":X,"training_score":[score],"cv_score":[cv_scores.mean()]}),ignore_index=True)
    return df

In [52]:
splitsSize = np.linspace(0.04,0.99,20)

0
10


In [ ]:
learning_df= myDT_LearningCurve(clf,X_Data,Y_Data,splitsSize,cv)
learning_df

In [ ]:
learning_df.describe()

In [ ]:

fig, ax = plt.subplots()
plt.plot(learning_df.percent, learning_df.training_score, color='blue', marker='o', markersize=5, label='training')

plt.plot(learning_df.percent, learning_df.cv_score, color='orange', marker='o', markersize=5, label='cross validation')

ax.legend()
ax.set_title("Learning Curve of CV:10")



####----------------------------------------------------------------
# Validation Curve
####----------------------------------------------------------------

In [35]:
depths = [1,2,4, 5, 8,10,15, 20,25,30,35,40,50,60,70,80,90,100,150,200]

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.validation_curve.html#sklearn.model_selection.validation_curve
from sklearn.model_selection import validation_curve
train_scores, valid_scores = validation_curve(DecisionTreeClassifier(),X_train, y_train, 
    param_name="max_depth", param_range=depths,
    cv=cv, n_jobs=1)


In [ ]:
train_scores

In [ ]:
valid_scores


In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
valid_scores_mean = np.mean(valid_scores, axis=1)
valid_scores_std = np.std(valid_scores, axis=1)

plt.title("Validation Curve of CV:10")
plt.xlabel("Depth")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2
plt.semilogx(depths, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(depths, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(depths, valid_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(depths, valid_scores_mean - valid_scores_std,
                 valid_scores_mean + valid_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()


####----------------------------------------------------------------
# Model Turning
####----------------------------------------------------------------

In [ ]:
learnig_df= myDT_ModelTurning(clf,depths,X_train,y_train,X_test,y_test,cv)
learnig_df

In [ ]:
learnig_df.describe()

In [ ]:

fig, ax = plt.subplots()
plt.plot(learnig_df.depth, learnig_df.training_score, color='blue', marker='o', markersize=5, label='training')

plt.plot(learnig_df.depth, learnig_df.cv_score, color='orange', marker='o', markersize=5, label='cross validation')

ax.legend()
ax.set_title("Training and Cross validation scores of CV:10")


<div class="alert alert-block alert-info">
Based on the plot, we saw that 
</div>


In [ ]:
#GridSearchCV
def myGS(X_train,y_train,depths,num_leafs):
    from sklearn.model_selection import GridSearchCV
    
    param_grid = [{'max_depth':depths,
                  'min_samples_leaf':num_leafs}]
    clf2 = DecisionTreeClassifier(random_state=0)
    gs = GridSearchCV(estimator=clf2, param_grid=param_grid, cv=10)
    
    gs = gs.fit(X_train, y_train)
    return gs
    

In [ ]:
depths = [1,2,4, 5, 8, 10,15, 20,25,30,35,40]
num_leafs = [1,2,4, 5, 8, 10, 20]
gs = myGS(X_train,y_train,depths,num_leafs)
#Training Score
gs.best_score_

In [ ]:
gs.best_params_
    

####----------------------------------------------------------------
# Final Evaluation
####----------------------------------------------------------------

In [ ]:
clf = DecisionTreeClassifier(random_state=0,max_depth=gs.best_params_['max_depth'],min_samples_leaf=gs.best_params_['min_samples_leaf'])
#Original Data
score,cv_scores,params = myDT(clf,X_train,y_train,X_test,y_test,cv)
score

In [ ]:
cv_scores.mean()

In [ ]:
params
